# lDE20 Validation Image Processing Notebook

See './Image_Analysis/Examples/1_Core_Image_Processing.ipynb' for a documented example of notebook usage.

#### Imports

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
# addition of active memory manager
import dask
dask.config.set({'distributed.scheduler.active-memory-manager.start': True});
dask.config.set({'distributed.scheduler.worker-ttl': "5m"});
dask.config.set({'distributed.scheduler.allowed-failures': 100});

dask_wd = "/home/de64/scratch/de64/dask"

# Part 1: Growth/Division

#### Specify Paths

In [ ]:
# experiment 1
headpath = "/home/de64/scratch/de64/sync_folder/2021-11-12_lDE20_Validation_1"
nd2file = "/home/de64/scratch/de64/sync_folder/2021-11-12_lDE20_Validation_1/Experiment.nd2"

# experiment 2
# headpath = "/home/de64/scratch/de64/sync_folder/2021-12-13_lDE20_Validation_2"
# nd2file = "/home/de64/scratch/de64/sync_folder/2021-12-13_lDE20_Validation_2/Experiment.nd2"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=100,
    n_workers_min=50,
    memory="4GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

##### Perform Extraction

In [ ]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    nd2file,
    headpath,
    tpts_per_file=50,
    ignore_fovmetadata=False,
    nd2reader_override={"z_levels": [], "z_coordinates": []},
)

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

#### List of paths to flat fields
 - /home/de64/scratch/de64/sync_folder/2021-10-21_Flat_Fields/MCHERRY_20x_Ph2_flatfield.tiff
 - /home/de64/scratch/de64/sync_folder/2022-12-26_Flat_Fields_Iris_9_40x/40x_DarkImage.tiff

In [ ]:
hdf5_extractor.inter_set_flatfieldpaths()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

## Kymographs

### Test Parameters



##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

After running the above line, you will have a running Dask client. Run the line below and click the link to supervise <br>the computation being administered by the scheduler. 

Don't be alarmed if the screen starts mostly blank, it may take time for your workers to spin up. If you get a 404 <br>error on a cluster, it is likely that your ports are not being forwarded properly. If this occurs, please register <br>the issue on github.

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, paramfile=True
)

##### Begin Kymograph Cropping 

In [ ]:
kymoclust.generate_kymographs(dask_controller)

In [ ]:
lane_overlay_handle = tr.lane_overlay(headpath,persist_data=True)
overlay = lane_overlay_handle.view_overlay(size=2500,vmin=0,vmax=4000)

In [ ]:
overlay

In [ ]:
lane_overlay_handle.save_rows()

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller,trench_timepoints_per_file=10000)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

## Fluorescence Segmentation

### Test Parameters

##### Initialize the interactive segmentation class

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
interactive_segmentation.choose_seg_channel_inter()

#### Import data

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=100,
    n_workers_min=20,
    memory="1GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath, paramfile=True)

In [ ]:
segment.dask_segment(dask_controller)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Lineage Tracing

### Test Parameters

In [ ]:
score_function = tr.tracking.scorefn(
    headpath,
    "fluorsegmentation",
    size_attr='axis_major_length',
    u_size=0.08,
    sig_size=0.03,
    u_pos=0.1,
    sig_pos=0.05,
    w_merge=0.,
)

In [ ]:
score_function.interactive_scorefn()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, "fluorsegmentation", ScoreFn=score_function, edge_limit=2,
)
data, orientation = score_function.output.result

In [ ]:
Tracking_Solver.interactive_tracking(data, orientation)

In [ ]:
Tracking_Solver.save_params()

### Generate Lineage Traces

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=300,
    n_workers_min=20,
    memory="1GB",
    death_timeout=60.,
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, 
    "fluorsegmentation", 
    paramfile=True, 
    size_estimation=True,
    size_estimation_method='Perimeter/Area',
    props_list=['area'], 
    props_to_unpack={},
    pixel_scaling_factors={'area': 2},
    intensity_props_list=['mean_intensity'] 
)

In [ ]:
Tracking_Solver.compute_all_lineages(dask_controller,entries_per_partition = 30000, overwrite=True)

#### Shutdown Dask

In [ ]:
dask_controller.shutdown()